In [2]:
#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6023581/
#https://journals.sagepub.com/doi/full/10.1177/0020294018813692

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import timeit
from numpy.fft import fft, fftfreq, ifft
from scipy.signal import welch


from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


In [4]:
#initialise start timer
start = timeit.default_timer()

#read in files as DFs
tr_time = pd.read_csv("train_time_series.csv")
tr_lab = pd.read_csv("train_labels.csv")

In [5]:
#Change DF columns
tr_time = tr_time.drop(columns = ["accuracy", "UTC time", "timestamp"]).rename(columns = {"Unnamed: 0":"measurement"})
tr_lab = pd.read_csv("train_labels.csv").rename(columns = {"Unnamed: 0":"measurement"}).drop(columns = ["UTC time", "timestamp"])

In [6]:
def accel_mag(accels):
    """takes an array containing x,y,z accelerations, calculates their magnitude"""
    return np.sqrt(np.sum(accels**2))

def rotation(a):
    """for roll use a=y, b=z; for pitch use a=x, b=z; for yaw use a=y, b=x"""
    return np.arctan(a)

#mean freq ignores the large peak inthe
def mean_freq(fft_vals, fft_freqs):
    """Calculates  the mean frequency of fourier transformed data"""
    return sum(fft_freqs[1 : int(len(fft_vals)/2+1)]*fft_vals[1 : int(len(fft_vals)/2+1)])/(sum(fft_vals[1 : int(len(fft_vals)/2+1)]))

In [7]:
#set sampling and calculate sampling rate for FFT
samp_int = 0.1 #in seconds
samp_rate = 1/samp_int #in hz or per sec


In [31]:
def calculate_features(tr_time, tr_lab):
    """Takes in the time series and labelled data and calculates features used for machine learning analysis"""
#overlapping windows of ~50%
#skip 1st value and last value, because of insufficient surrounding data.
for i in tr_lab.measurement[1:374]:
    boolinds = pd.Series((tr_time.measurement.values <= (i + 9)) & (tr_time.measurement.values > previous))
    frame = tr_time[boolinds.values]
    
    #calculate magnitude of acceleration and variation of acceleration for the step - maybe delete later    
    accels = frame.loc[:,("x","y","z")]
    mags = accels.apply(accel_mag, axis="columns")
    
    
    #calculate rolls, pitch, yaws
    rolls = rotation([accels.y, accels.z])
    pitches = rotation([accels.x, accels.z])
    yaws = rotation([accels.y, accels.x])
    
    #mean mag
    tr_lab.loc[tr_lab.measurement == i, "mean_accel_mag"] = np.mean(mags)
    #SD mag
    tr_lab.loc[tr_lab.measurement == i, "SD_mag"] = np.std(mags)
    #mag var
    tr_lab.loc[tr_lab.measurement == i, "var_mag"] = np.var(mags)
    #Coeff of Variation / Relative SD
    tr_lab.loc[tr_lab.measurement == i, "RSD_mag"] = np.std(mags)/np.mean(mags)
    #mag min
    tr_lab.loc[tr_lab.measurement == i, "min_mag"] = np.amin(mags)
    #mag max
    tr_lab.loc[tr_lab.measurement == i, "max_mag"] = np.amax(mags)
    # mag 25, 50, 75 percentile
    tr_lab.loc[tr_lab.measurement == i, "per_25_mag"] = np.percentile(mags,25)
    tr_lab.loc[tr_lab.measurement == i, "per_50_mag"] = np.percentile(mags,50)
    tr_lab.loc[tr_lab.measurement == i, "per_75_mag"] = np.percentile(mags,75)
    #mean rotations
    tr_lab.loc[tr_lab.measurement == i, "mean_roll"] = np.mean(rolls)
    tr_lab.loc[tr_lab.measurement == i, "mean_pitch"] = np.mean(pitches)
    tr_lab.loc[tr_lab.measurement == i, "mean_yaw"] = np.mean(yaws)
    #SD rotations
    tr_lab.loc[tr_lab.measurement == i, "SD_roll"] = np.std(rolls)
    tr_lab.loc[tr_lab.measurement == i, "SD_pitch"] = np.std(pitches)
    tr_lab.loc[tr_lab.measurement == i, "SD_yaw"] = np.std(yaws)

    #Fourier transform each step
    n = len(mags)
    mags_windowed = np.hanning(len(mags))*mags
    f = np.linspace(0, samp_rate, n)
    fft_vals = np.abs(fft(mags_windowed))
    tr_lab.loc[tr_lab.measurement == i, "mean_freq"] = mean_freq(fft_vals,f)
    tr_lab.loc[tr_lab.measurement == i, "max_intensity"] = np.max(fft_vals[1:])

    #calculate power spectrum density 
    f, psd = welch(mags, fs=samp_rate)
    #find index corresponding to max power between 0.3 and 0.8Hz
    #find max p
    tr_lab.loc[tr_lab.measurement == i, "max_power"] = np.max(psd[(f > 0.3) & (psd < 0.8)])
    #find frequency of max power
    tr_lab.loc[tr_lab.measurement == i, "max_p_freq"] = f[np.argmax(psd)]
    #total power in signal
    tr_lab.loc[tr_lab.measurement == i, "power"] = np.trapz(psd, f) 

    previous = i

#drop unused labelled data.
tr_lab = tr_lab.drop([tr_lab.index[0], tr_lab.index[len(tr_lab) - 1]])

In [85]:
covariates = tr_lab.drop(columns = ['measurement', 'label']).columns

y_train = tr_lab['label']
X_train = tr_lab[covariates]
X_train

,mean_accel_mag,SD_mag,var_mag,RSD_mag,min_mag,max_mag,per_25_mag,per_50_mag,per_75_mag,mean_roll,mean_pitch,mean_yaw,SD_roll,SD_pitch,SD_yaw,mean_freq,max_intensity,max_power,max_p_freq,power
3,1.008786,0.072167,0.005208,0.071538,0.852482,1.251372,0.991498,1.011016,1.020965,-0.350190,0.049377,-0.382059,0.439998,0.104720,0.404243,0.978772,10.743448,0.002786,4.761905,0.003735
4,1.007379,0.033143,0.001098,0.032900,0.902399,1.101590,1.004853,1.007719,1.010227,-0.391321,0.026291,-0.368093,0.398948,0.074305,0.419572,0.687091,4.880440,0.000048,4.736842,0.000076
5,1.005584,0.008227,0.000068,0.008182,0.987280,1.027750,1.002109,1.006927,1.009144,-0.381773,0.018989,-0.384782,0.408277,0.069802,0.402284,0.657311,4.865574,0.000024,1.052632,0.000054
6,1.022201,0.148611,0.022085,0.145383,0.817667,1.587389,0.991007,1.004542,1.011859,-0.303981,0.082442,-0.395256,0.485152,0.121084,0.390095,0.815231,4.910824,0.000855,2.105263,0.001685
7,1.022756,0.149196,0.022259,0.145876,0.817667,1.587389,0.992556,1.003677,1.027276,-0.298319,0.112028,-0.367590,0.490582,0.126677,0.415695,1.758800,5.195433,0.016970,2.631579,0.043733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,1.211536,0.662540,0.438959,0.546859,0.409712,3.168469,0.843505,0.990541,1.502084,-0.319991,0.132011,-0.290370,0.501787,0.369147,0.567861,1.987153,5.628007,0.176071,1.578947,0.223237
368,1.262138,0.583541,0.340520,0.462343,0.409712,2.422324,0.912244,1.144645,1.825492,-0.389212,0.040098,-0.351020,0.466605,0.378735,0.562260,1.918741,6.032479,0.107480,1.578947,0.159763
369,1.270479,0.660060,0.435679,0.519536,0.434802,3.079310,0.897654,1.144645,1.590064,-0.379902,0.102069,-0.307076,0.472148,0.362126,0.593597,1.907556,6.469696,0.136777,1.578947,0.219085
370,1.222234,0.592357,0.350887,0.484651,0.468032,3.079310,0.913894,1.034177,1.422695,-0.403515,0.047773,-0.331108,0.438561,0.366807,0.564350,2.249063,7.504650,0.194595,3.684211,0.599879


In [86]:
# pca = PCA(n_components='mle')
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# print("Reduced shape: {}".format(str(X_train.shape)))

In [87]:
# sel = VarianceThreshold(threshold=(.1))
# X_train = sel.fit_transform(tr_lab[covariates])
# X_train.shape

In [93]:
#Initialise a random forest and determine best set of hyperparameters
rfc = RandomForestClassifier()

parameters = {
    "n_estimators":[5,10,50,100,250],
    "max_depth":[2,4,8,16,32,None]   
}

cv = GridSearchCV(rfc,parameters,cv=10)
cv.fit(X_train, y_train)
best_params = cv.best_params_ 
best_params

{'max_depth': None, 'n_estimators': 100}

In [94]:
#initialise random forest model with best hyperparameters and train
rfc = RandomForestClassifier(max_depth = best_params['max_depth'], n_estimators = best_params['n_estimators'])
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [99]:
np.mean(cross_val_score(rfc,X_train, y_train, cv=5))

0.7561273602369492

In [69]:
#read in files as DFs
te_time = pd.read_csv("test_time_series.csv")
te_lab = pd.read_csv("test_labels.csv")

In [70]:
stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  677.2722674


In [ ]:
#try a KNN
knn = Nearest Classifier()

parameters = {
    "n_estimators":[5,10,50,100,250],
    "max_depth":[2,4,8,16,32,None]   
}

cv = GridSearchCV(rfc,parameters,cv=10)
cv.fit(X_train, y_train)
best_params = cv.best_params_ 
best_params

In [ ]:
#Standardise X,Y,Z accels
std = preprocessing.StandardScaler()
for k in ["x", "y", "z"]:
    tr_time[k] = std.fit_transform(tr_time[k].values.reshape(-1,1))
tr_time.tail()

array([0.0056515 , 0.0025298 , 0.01139067, 0.07704185, 0.09652372,
       0.05106674, 0.06748851, 0.08006489, 0.10602767, 0.0209424 ])

In [24]:
f[]

array([0.        , 0.52631579, 1.05263158, 1.57894737, 2.10526316,
       2.63157895, 3.15789474, 3.68421053, 4.21052632, 4.73684211])

In [30]:
np.argmax(psd)


8

In [482]:
tr_lab

,measurement,label,mean_accel_mag,SD_mag,var_mag,RSD_mag,min_mag,max_mag,per_25_mag,per_50_mag,...,mean_pitch,mean_yaw,SD_roll,SD_pitch,SD_yaw,mean_freq,max_intensity,max_power,max_p_freq,power
2,20609,1,1.001965,0.065664,0.004312,0.065535,0.852482,1.208477,0.989259,1.011607,...,0.047209,-0.380767,0.437295,0.107240,0.401804,1.245041,8.110292,10.0,0.0,0.004188
3,20619,1,1.014013,0.080461,0.006474,0.079349,0.884094,1.251372,0.996758,1.011652,...,0.050925,-0.385778,0.443279,0.098584,0.403923,1.560199,4.905462,8.0,0.0,0.012689
4,20629,1,1.007379,0.033143,0.001098,0.032900,0.902399,1.101590,1.004853,1.007719,...,0.026291,-0.368093,0.398948,0.074305,0.419572,0.687091,4.880440,8.0,0.0,0.000076
5,20639,1,1.005584,0.008227,0.000068,0.008182,0.987280,1.027750,1.002109,1.006927,...,0.018989,-0.384782,0.408277,0.069802,0.402284,0.657311,4.865574,1.0,0.0,0.000054
6,20649,1,1.022201,0.148611,0.022085,0.145383,0.817667,1.587389,0.991007,1.004542,...,0.082442,-0.395256,0.485152,0.121084,0.390095,0.815231,4.910824,3.0,0.0,0.001685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,24269,4,1.262138,0.583541,0.340520,0.462343,0.409712,2.422324,0.912244,1.144645,...,0.040098,-0.351020,0.466605,0.378735,0.562260,1.918741,6.032479,2.0,0.0,0.159763
369,24279,4,1.270479,0.660060,0.435679,0.519536,0.434802,3.079310,0.897654,1.144645,...,0.102069,-0.307076,0.472148,0.362126,0.593597,1.907556,6.469696,2.0,0.0,0.219085
370,24289,4,1.222234,0.592357,0.350887,0.484651,0.468032,3.079310,0.913894,1.034177,...,0.047773,-0.331108,0.438561,0.366807,0.564350,2.249063,7.504650,6.0,0.0,0.599879
371,24299,4,1.282362,0.555392,0.308460,0.433101,0.408598,2.796540,0.934651,1.131752,...,0.016340,-0.344778,0.430674,0.420648,0.589646,2.294548,6.585685,4.0,0.0,0.542177


In [177]:
psd = welch(mags, fs=samp_rate)
psd

(array([0.        , 0.43478261, 0.86956522, 1.30434783, 1.73913043,
        2.17391304, 2.60869565, 3.04347826, 3.47826087, 3.91304348,
        4.34782609, 4.7826087 ]),
 array([0.00075746, 0.00650139, 0.0074904 , 0.00841048, 0.08604953,
        0.03825272, 0.02212067, 0.040107  , 0.01621776, 0.07028393,
        0.04486184, 0.00503467]))

In [178]:
psd_max = np.argmax(a[1])
psd_max

1

In [156]:
psd[0][psd_max]

0.4347826086956521

In [158]:
np.argmax(psd[1][(psd[0] > 0.3) & (psd[0] < 0.8)])

0

In [252]:
#calculate power spectrum density 
f, psd = welch(mags, fs=samp_rate)
#find index corresponding to max power between 0.3 and 0.8Hz
#tr_lab.loc[tr_lab.measurement == i, "max_power"] = np.argmax(psd[1][(psd[0] > 0.3) & (psd[0] < 0.8)])
#find frequency of max power
#tr_lab.loc[tr_lab.measurement == i, "mean_freq"] = psd[0][max_p_ind]    
f

array([0.        , 0.52631579, 1.05263158, 1.57894737, 2.10526316,
       2.63157895, 3.15789474, 3.68421053, 4.21052632, 4.73684211])

In [199]:
freq_int = f[1] - f[0]
np.trapz(psd, f)

0.8011948603379807

In [253]:
    #Fourier transform each step
    n = len(mags)
    mags_windowed = np.hanning(len(mags))*mags
    f = np.linspace(0, samp_rate, n)
    fft_vals = np.abs(fft(mags_windowed))
    tr_lab.loc[tr_lab.measurement == i, "mean_freq"] = mean_freq(fft_vals,f)
    fft_vals

array([9.9371499 , 5.34976885, 0.76353909, 1.42142026, 1.63783038,
       1.05222463, 1.21622593, 1.33346806, 1.63741016, 0.60578641,
       0.60578641, 1.63741016, 1.33346806, 1.21622593, 1.05222463,
       1.63783038, 1.42142026, 0.76353909, 5.34976885])